# 라이브러리 import 및 GPU 설정

In [1]:
import torch
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import glob
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/cnn_practice/

/content/drive/MyDrive/cnn_practice


In [4]:
!pwd

/content/drive/MyDrive/cnn_practice


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# 이미지데이터 전처리

In [6]:
# 디렉토리 설정
base_dir = './datasets/'
categories = ['cats', 'dogs', 'monkeys']
nb_classes = len(categories)

In [7]:
# 이미지 크기 지정
image_w = 200
image_h = 200
pixels = image_w*image_h*3

In [8]:
# 이미지데이터 읽어들이기
X = []
Y = []

for idx, category in enumerate(categories):
    
    # 이미지
    image_dir = base_dir+category+'/*'
    files = glob.glob(image_dir)
    
    for f in files:
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))

        # 이미지를 RGB로 각각 쪼개기
        r, g, b = img.split()

        # 이미지 정규화
        r_img = np.asarray(np.float32(r)/255.0)
        g_img = np.asarray(np.float32(g)/255.0)
        b_img = np.asarray(np.float32(b)/255.0)

        rgb_img = np.asarray([r_img, g_img, b_img])

        # 가공한 이미지 추가
        X.append(rgb_img)
        Y.append(idx)

In [9]:
import pandas as pd
pd.DataFrame(X[0][0]).shape

(200, 200)

In [10]:
from torch.utils.data import TensorDataset, DataLoader

X = np.array(X, dtype='float32')
Y = np.array(Y, dtype='int64')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=32, shuffle=True)

In [11]:
total_batch = len(train_loader)

# 모델 설계 및 훈련

In [12]:
learning_rate = 0.001
training_epochs = 30
batch_size = 32

In [13]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.3))

        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            torch.nn.Dropout(0.3))

        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            torch.nn.Dropout(0.5))

        self.layer4 = torch.nn.Sequential(
            torch.nn.Dropout(0.2))
        
        self.fc1 = torch.nn.Linear(36864, 128, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        
        self.layer5 = torch.nn.Sequential(
            torch.nn.Dropout(0.5))
        
        self.fc3 = torch.nn.Linear(128, nb_classes, bias=True)
        torch.nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.layer4(out)
        out = self.fc1(out)
        out = self.layer5(out)
        out = self.fc3(out)
        return out

In [14]:
# CNN 모델 정의
model = CNN().to(device)

In [15]:
from torchsummary import summary as summary_
summary_(model, (3,200,200),batch_size=32)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 32, 198, 198]             896
              ReLU-2         [32, 32, 198, 198]               0
         MaxPool2d-3           [32, 32, 99, 99]               0
           Dropout-4           [32, 32, 99, 99]               0
            Conv2d-5           [32, 32, 97, 97]           9,248
              ReLU-6           [32, 32, 97, 97]               0
         MaxPool2d-7           [32, 32, 48, 48]               0
           Dropout-8           [32, 32, 48, 48]               0
            Conv2d-9           [32, 64, 46, 46]          18,496
             ReLU-10           [32, 64, 46, 46]               0
        MaxPool2d-11           [32, 64, 24, 24]               0
          Dropout-12           [32, 64, 24, 24]               0
          Dropout-13                [32, 36864]               0
           Linear-14                  [

In [16]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
from torch.autograd import Variable
for epoch in range(training_epochs):
    total_loss = 0

    for train_x, train_y in train_loader:
      train_x, train_y = Variable(train_x.float().cuda()), Variable(train_y.long().cuda())
      optimizer.zero_grad()
      output = model(train_x)
      loss = criterion(output, train_y)
      loss.backward()
      optimizer.step()
      total_loss += loss.data.item()
      if (epoch+1) % 10 == 0:
        print(epoch+1, total_loss)

test_x, test_y = Variable(X_test.float().cuda()), Variable(y_test.long().cuda())
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
accuracy

10 0.2588033378124237
10 0.7379087507724762
10 0.9148424118757248
10 1.0996987521648407
10 1.5724490880966187
10 1.7572866380214691
10 1.9332176446914673
10 2.0578571557998657
10 2.202564299106598
10 2.3078015074133873
10 2.575066201388836
10 2.887410692870617
10 2.988998770713806
10 3.291136711835861
10 3.6685485541820526
10 3.763991855084896
10 3.934601478278637
10 4.120675005018711
10 4.2809473648667336
10 4.445634908974171
10 4.627551294863224
10 4.97964421659708
10 5.2898129895329475
10 5.347398366779089
10 5.58385531231761
10 5.636936526745558
10 5.872961919754744
10 6.074702467769384
10 6.19639740511775
10 6.357655834406614
10 6.6257720701396465
10 6.743769947439432
10 6.88478023186326
10 7.266673360019922
10 7.350750777870417
10 7.543833065778017
10 7.788581032305956
10 7.930673141032457
10 8.322402257472277
10 8.436074662953615
10 8.903923530131578
10 9.024612333625555
10 9.167332824319601
10 9.266266617923975
10 9.459778133779764
10 9.651765365153551
10 9.870732117444277
10 1

RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), './')